In [1]:
import torch
from ResNet.ResNet import ResNet18

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = ResNet18(out_neurons=1).to(device)
resnet18.load_state_dict(
    torch.load(
        r"..\results\ResNet-18\best_net.pth",
        map_location=device,
        weights_only=True,
    )
)

<All keys matched successfully>

In [2]:
from ResNet.dataloader import data_loader

data_dir = r"../data"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

out_neurons = 1
num_epochs = 100
batch_size = 64
learning_rate = 0.0001
patience = 9

_, _, mean, std = data_loader(
    data_dir=data_dir,
    batch_size=batch_size,
    num_workers=4,
    pin_memory=True,
)
print(mean)
print()
print(std)

[0.5642121434211731, 0.5642121434211731, 0.5642121434211731]

[0.23441407084465027, 0.23441407084465027, 0.23441407084465027]


In [3]:
from torchvision import transforms
from PIL import Image


def predict(model, image_path, device="cpu", img_shape=(224, 224), threshold=0.1):
    image = Image.open(image_path).convert("RGB")

    transform = transforms.Compose(
        [
            transforms.Resize(img_shape),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.5642121434211731, 0.5642121434211731, 0.5642121434211731],
                std=[0.23441407084465027, 0.23441407084465027, 0.23441407084465027],
            ),
        ]
    )

    image = transform(image).unsqueeze(0)
    image = image.to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(image)
        probabilities = torch.sigmoid(outputs)
        predicted = (probabilities > threshold).int()

    if predicted[0][0] == 1:
        return "Defective", round(float(probabilities[0][0]), 2) * 100

    return "Non-Defective", round(1 - float(probabilities[0][0]), 2) * 100

In [5]:
image_path_def = r'..\data\train\def_front\cast_def_0_2.jpeg'
image_path_ok = r'..\data\train\ok_front\cast_ok_0_9.jpeg'
prediction, propability = predict(resnet18, image_path_def)
print(f"Predicted class: {prediction} -> %{propability}")

Predicted class: Defective -> %100.0
